# Census APIs

Request a key at <https://api.census.gov/data/key_signup.html>

An email will be sent to the addres you provide. The email will include a link to activate the key. It may take a few minutes before the key can be activated.

In [1]:
# Load API key

# Note the file with the API key is not in the GitHub repo.
# Keys need to be read from a local directory
with open("jwCensusApi", "r") as i:
    key = i.read()

In [2]:
import json
import requests
import pandas as pd
import matplotlib as plt

In [3]:
'''
Current Population Survey Annual Social and Economic Supplement
https://api.census.gov/data.html

Vars: https://api.census.gov/data/2021/cps/asec/mar/variables.html
'''

endpoint = "http://api.census.gov/data/2021/cps/asec/mar" # state geographies
ex1_url = "https://api.census.gov/data/2021/cps/asec/mar?get=NOW_COV,MARSUPWT,A_MARITL&for=state:35&A_HGA=39"
ex2_url = "https://api.census.gov/data/2021/cps/asec/mar?tabulate=weight(MARSUPWT)&col+NOW_COV&row+A_MARITL&for=state:35&A_HGA=39"

In [23]:
api_request_url = ex1_url + "&key=" + key
api_request = requests.get(api_request_url)

In [24]:
api_json = api_request.json()

In [26]:
# print(json.dumps(api_json, indent=4)) # long output

### Q: How to get info on variables via API?

We can look it up online, but we may also get this info dynamically.

In [7]:
# Maybe create dictionaries of vars:defs
# populate via iteration

var = "NOW_COV"
var_def_url = endpoint + "/variables/" + var

In [8]:
var_def_request = requests.get(var_def_url)

In [9]:
var_json = var_def_request.json()

In [10]:
print(json.dumps(var_json, indent=4))

{
    "name": "NOW_COV",
    "label": "Currently covered by health insurance coverage",
    "predicateType": "int",
    "group": "N/A",
    "limit": 0,
    "suggested-weight": "MARSUPWT",
    "values": {
        "item": {
            "2": "No",
            "1": "Yes"
        }
    }
}


In [11]:
'''
Variables of interest

A_HGA - educational attainment
AGE1 - age
AGI - federal adjusted gross income

NM FIPS ID = 35
'''

vars_list = ["A_HGA", "AGE1", "AGI"]
vars_dict = {}

for v in vars_list:
    var_def_url = endpoint + "/variables/" + v
    var_def_request = requests.get(var_def_url)
    var_json = var_def_request.json()
    vars_dict[v] = var_json

In [12]:
vars_dict

{'A_HGA': {'name': 'A_HGA',
  'label': 'Demographics, Educational attainment',
  'predicateType': 'int',
  'group': 'N/A',
  'limit': 0,
  'suggested-weight': 'MARSUPWT',
  'values': {'item': {'35': '9th Grade',
    '41': 'Assc degree-occupation/vocation',
    '0': 'Children',
    '31': 'Less Than 1st Grade',
    '44': "Master's degree (MA,MS,MENG,MED,MSW,MBA)",
    '43': "Bachelor's degree (BA,AB,BS)",
    '34': '7th and 8th grade',
    '38': '12th Grade No Diploma',
    '37': '11th Grade',
    '33': '5th Or 6th Grade',
    '45': 'Professional school degree (MD,DDS,DVM,L',
    '39': 'High school graduate-high school diploma',
    '32': '1st,2nd,3rd,or 4th grade',
    '40': 'Some College But No Degree',
    '46': 'Doctorate degree (PHD,EDD)',
    '36': '10th Grade',
    '42': 'Assc degree-academic program'}}},
 'AGE1': {'name': 'AGE1',
  'label': 'Demographics, Age recode, persons 15+ years',
  'predicateType': 'int',
  'group': 'N/A',
  'limit': 0,
  'suggested-weight': 'MARSUPWT',
  

In [17]:
# API expects vars to be comma separated, no spaces

vars_query = ','.join(str(v) for v in vars_list)
print(vars_query)

A_HGA,AGE1,AGI


In [28]:
# request non-tabulated data
# URL format: "https://api.census.gov/data/2021/cps/asec/mar?get=NOW_COV,MARSUPWT,A_MARITL&for=state:35"

# We can parameterize the state but for now we will hard code all and NM (35)
api_request_url = endpoint + "?get=" + vars_query + "&for=state:*&key=" + key
api_request = requests.get(api_request_url)
api_json = api_request.json()

In [29]:
type(api_json) # results seem to generally be lists, but check in case a dictionary is returned

list

In [43]:
census_df = pd.read_json(json.dumps(api_json), typ = "frame", orient = "values")

In [45]:
census_df.head()

,0,1,2,3
0,A_HGA,AGE1,AGI,state
1,39,12,10000,23
2,39,12,0,23
3,39,17,0,23
4,43,15,80819,23


In [46]:
census_df.columns = census_df.iloc[0]

In [47]:
census_df.head()

,A_HGA,AGE1,AGI,state
0,A_HGA,AGE1,AGI,state
1,39,12,10000,23
2,39,12,0,23
3,39,17,0,23
4,43,15,80819,23


In [49]:
census_df = census_df.drop(census_df.index[0])

In [50]:
census_df.head()

,A_HGA,AGE1,AGI,state
1,39,12,10000,23
2,39,12,0,23
3,39,17,0,23
4,43,15,80819,23
5,39,15,0,23


In [51]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163543 entries, 1 to 163543
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   A_HGA   163543 non-null  object
 1   AGE1    163543 non-null  object
 2   AGI     163543 non-null  object
 3   state   163543 non-null  object
dtypes: object(4)
memory usage: 6.2+ MB


In [52]:
# We can separately request data for just NM, or we can subset
# recall that the FIPS code for NM is 35

nm_census_data = census_df[census_df["state"] == "35"].copy() # quotes around "35" b/c it is an object not an int

In [53]:
nm_census_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2900 entries, 126038 to 128937
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A_HGA   2900 non-null   object
 1   AGE1    2900 non-null   object
 2   AGI     2900 non-null   object
 3   state   2900 non-null   object
dtypes: object(4)
memory usage: 113.3+ KB


In [54]:
# out of curiosity let's get the NM data via API
# same as before with one different parameter

api_request_url = endpoint + "?get=" + vars_query + "&for=state:35&key=" + key
api_request = requests.get(api_request_url)
api_json = api_request.json()

nm_census_df = pd.read_json(json.dumps(api_json), typ = "frame", orient = "values")
nm_census_df.columns = nm_census_df.iloc[0]
nm_census_df = nm_census_df.drop(nm_census_df.index[0])

In [55]:
nm_census_df.info() # the data have the same number of rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2900 entries, 1 to 2900
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A_HGA   2900 non-null   object
 1   AGE1    2900 non-null   object
 2   AGI     2900 non-null   object
 3   state   2900 non-null   object
dtypes: object(4)
memory usage: 113.3+ KB


In [66]:
# Let's make a function to get the data and convert to data frame
def get_census_data(endpoint_url, vars_list, key, state = "*"):
    # First, convert vars_list to string
    vars_query = ','.join(str(v) for v in vars_list)
    
    # Build the request URL
    api_request_url = endpoint_url + "?get=" + vars_query + "&for=state:" + state + "&key=" + key
    
    # Request the data
    api_request = requests.get(api_request_url)
    api_json = api_request.json()
    
    # Convert returned JSON to dataframe
    df = pd.read_json(json.dumps(api_json), typ = "frame", orient = "values")
    
    # Use first row values for col headers
    df.columns = df.iloc[0]
    
    # Drop first row, which is still header info
    df = df.drop(df.index[0])
    
    return df

# While we're at it, create a function to get var defs
# No key needed
def get_census_vars(vars_list):
    vars_dict = {}
    for v in vars_list:
        var_def_url = endpoint + "/variables/" + v
        var_def_request = requests.get(var_def_url)
        var_json = var_def_request.json()
        vars_dict[v] = var_json
    return vars_dict

In [71]:
# Now we can make ad hoc requests
# but we will redo our initial request

vars_list = ["A_HGA", "AGE1", "AGI"]
var_info = get_census_vars(vars_list)
census_data = get_census_data(endpoint, vars_list, key) # use default * for all state data

In [72]:
census_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163543 entries, 1 to 163543
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   A_HGA   163543 non-null  object
 1   AGE1    163543 non-null  object
 2   AGI     163543 non-null  object
 3   state   163543 non-null  object
dtypes: object(4)
memory usage: 6.2+ MB


In [69]:
# try again with just NM
# census_data = get_census_data(endpoint, vars_list, key, state = "35") # NM
# census_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2900 entries, 1 to 2900
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A_HGA   2900 non-null   object
 1   AGE1    2900 non-null   object
 2   AGI     2900 non-null   object
 3   state   2900 non-null   object
dtypes: object(4)
memory usage: 113.3+ KB


In [70]:
# Things work - let's see about reproducing stats from the press release
# Right now we have age and education attainment (and income) for 2021

'''
In 2021, the highest level of education of the population age 25 and older in the United States 
was distributed as follows: 

    8.9% had less than a high school diploma or equivalent.
    27.9% had high school graduate as their highest level of school completed. 
    14.9% had completed some college but not a degree.
    10.5% had an associate degree as their highest level of school completed.
    23.5% had a bachelor’s degree as their highest degree.
    14.4% had completed an advanced degree such as a master’s degree, professional degree or doctoral degree. 
'''

var_info

{'A_HGA': {'name': 'A_HGA',
  'label': 'Demographics, Educational attainment',
  'predicateType': 'int',
  'group': 'N/A',
  'limit': 0,
  'suggested-weight': 'MARSUPWT',
  'values': {'item': {'35': '9th Grade',
    '41': 'Assc degree-occupation/vocation',
    '0': 'Children',
    '31': 'Less Than 1st Grade',
    '44': "Master's degree (MA,MS,MENG,MED,MSW,MBA)",
    '43': "Bachelor's degree (BA,AB,BS)",
    '34': '7th and 8th grade',
    '38': '12th Grade No Diploma',
    '37': '11th Grade',
    '33': '5th Or 6th Grade',
    '45': 'Professional school degree (MD,DDS,DVM,L',
    '39': 'High school graduate-high school diploma',
    '32': '1st,2nd,3rd,or 4th grade',
    '40': 'Some College But No Degree',
    '46': 'Doctorate degree (PHD,EDD)',
    '36': '10th Grade',
    '42': 'Assc degree-academic program'}}},
 'AGE1': {'name': 'AGE1',
  'label': 'Demographics, Age recode, persons 15+ years',
  'predicateType': 'int',
  'group': 'N/A',
  'limit': 0,
  'suggested-weight': 'MARSUPWT',
  

In [76]:
# First subset to AGE1 values >= 6 (25 and older)

# convert cols to int
census_data = census_data.astype(int)
census_data.info()
age_gte25 = census_data[census_data["AGE1"] >= 6].copy()
age_gte25.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163543 entries, 1 to 163543
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   A_HGA   163543 non-null  int32
 1   AGE1    163543 non-null  int32
 2   AGI     163543 non-null  int32
 3   state   163543 non-null  int32
dtypes: int32(4)
memory usage: 3.7 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 108998 entries, 1 to 163543
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   A_HGA   108998 non-null  int32
 1   AGE1    108998 non-null  int32
 2   AGI     108998 non-null  int32
 3   state   108998 non-null  int32
dtypes: int32(4)
memory usage: 2.5 MB


In [78]:
age_gte25.head()

,A_HGA,AGE1,AGI,state
1,39,12,10000,23
2,39,12,0,23
3,39,17,0,23
4,43,15,80819,23
5,39,15,0,23


In [91]:
var_info

{'A_HGA': {'name': 'A_HGA',
  'label': 'Demographics, Educational attainment',
  'predicateType': 'int',
  'group': 'N/A',
  'limit': 0,
  'suggested-weight': 'MARSUPWT',
  'values': {'item': {'35': '9th Grade',
    '41': 'Assc degree-occupation/vocation',
    '0': 'Children',
    '31': 'Less Than 1st Grade',
    '44': "Master's degree (MA,MS,MENG,MED,MSW,MBA)",
    '43': "Bachelor's degree (BA,AB,BS)",
    '34': '7th and 8th grade',
    '38': '12th Grade No Diploma',
    '37': '11th Grade',
    '33': '5th Or 6th Grade',
    '45': 'Professional school degree (MD,DDS,DVM,L',
    '39': 'High school graduate-high school diploma',
    '32': '1st,2nd,3rd,or 4th grade',
    '40': 'Some College But No Degree',
    '46': 'Doctorate degree (PHD,EDD)',
    '36': '10th Grade',
    '42': 'Assc degree-academic program'}}},
 'AGE1': {'name': 'AGE1',
  'label': 'Demographics, Age recode, persons 15+ years',
  'predicateType': 'int',
  'group': 'N/A',
  'limit': 0,
  'suggested-weight': 'MARSUPWT',
  

In [84]:
# convert A_HGA, AGE1, and state to categories
age_gte25.A_HGA = age_gte25.A_HGA.astype("category")
age_gte25.AGE1 = age_gte25.AGE1.astype("category")
age_gte25.state = age_gte25.state.astype("category")

In [85]:
age_gte25.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108998 entries, 1 to 163543
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   A_HGA   108998 non-null  category
 1   AGE1    108998 non-null  category
 2   AGI     108998 non-null  int32   
 3   state   108998 non-null  category
dtypes: category(3), int32(1)
memory usage: 1.6 MB


In [89]:
type(age_gte25.iloc[0, 0])

numpy.int64

In [90]:
age_gte25.iloc[0, 0]

39

In [93]:
# so cols are categories but cell values are still int (?)

# Educational attainment: <= 38 less than HS diploma or eq
# 39 == HS diploma, 40 == some college ND, 41 & 42 == assoc Deg, 43 == Bachelor's, 
# 44 & 45 Masters or professional degree, 46 == doctorate

# there is probably a faster way to do this
ed_groups = age_gte25.groupby("A_HGA")

In [109]:
#ed_groups.groups

In [97]:
# 27.9% HS grads per press release
hs_grads = ed_groups.get_group(39)

In [102]:
len(hs_grads)

30656

In [103]:
(30656 / 108998) * 100

28.1252867025083

In [104]:
# 14.9% had some college

some_college_nd = ed_groups.get_group(40)

In [106]:
len(some_college_nd)

16369

In [107]:
(16369/108998) * 100

15.017706746912788

In [112]:
# 31-38 for no HS
# 8.9% report less than HS diploma or eq

no_diploma = 0
for i in range(31, 39):
    g = ed_groups.get_group(i)
    no_diploma += len(g)

In [113]:
no_diploma

10289

In [114]:
(10289/108998) * 100

9.439622745371475